# LOLC Securities Limited


In [1]:
import numpy as np
import pandas as pd

## Data import

In [2]:
from pathlib import Path
from Notebooks.scripts.excel_utils import read_excel_table

# Notebook typically runs from the Notebooks directory; support both root and Notebooks as CWD
root = Path.cwd().resolve()
if root.name == 'Notebooks':
    root = root.parent
lse_dir = root / 'Data' / 'LSE'

frames = {
    p.name: read_excel_table(str(p))
    for p in lse_dir.iterdir()
    if p.suffix.lower() in {'.xlsx', '.xls'}
}

# `frames` is a dict: filename -> DataFrame


In [3]:
system_user = frames['BBO 1 List of System Users.xls']
advisor_maintenance_act = frames['BBO 2 Advisor Maintenance activities.xlsx']
user_maintenance_act = frames['BBO 3 User Maintenance activities.xlsx']
client_details = frames['BBO 4 Client Details Report.xlsx']
login_history = frames['BBO 5 Login History.xls']
user_management_act = frames['OMS 2 User Management Activities.xlsx']
cds_acc_association = frames['OMS 3 CDS Account Association.xlsx']
advisor_code_mapping = frames['OMS 4 Advisor Code Mapping.xlsx']
password_changes = frames['OMS 5 Password Changes.xlsx']

## System User Data

Check if all the users given in the dataset are unique and there are no missing values.

In [4]:
login_ids = system_user['Login ID']

# Case-insensitive unique count (ignores NA)
unique_ci = login_ids.astype('string').str.strip().str.lower().nunique(dropna=True)
print(unique_ci)

# NA count
na_count = login_ids.isna().sum()
print(na_count)
print(np.shape(system_user))

87
0
(87, 13)


In [5]:
#Filter out the inactive users
active_users = system_user[system_user['Status'] == 'Active']

In [10]:
date_col = 'Last Password Change Date'  # use your actual column name
last_pass = active_users.copy()

# Parse dates
last_pass[date_col] = pd.to_datetime(last_pass[date_col], errors='coerce')

today = pd.Timestamp.today().normalize()
age_days = (today - last_pass[date_col]).dt.days

# Buckets: <= 6 months, 6–12 months, > 12 months
bins = [-1, 182, 365, np.inf]
labels = [
    'Changed in last 6 months',
    'No change 6–12 months',
    'No change >12 months',
]

# Categorize
last_pass['Password Category'] = pd.cut(age_days, bins=bins, labels=labels)

# Allow adding a new category for missing dates, then fill it
last_pass['Password Category'] = (
    last_pass['Password Category']
    .cat.add_categories(['Missing password change date'])
    .fillna('Missing password change date')
)

# Final dataframe with requested columns
password_change = last_pass[['Login ID', 'Name', date_col, 'Password Category']]